# Проект по SQL

## Цель исследования:
Проанализировать базу данных крупного сервиса для чтения книг по подписке, чтобы сформулировать ценностное предложение для нового продукта

## Выведем первые строки каждой из таблиц

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])


In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
pd.io.sql.read_sql('''SELECT * 
                     FROM books
                     LIMIT 5''', con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
pd.io.sql.read_sql('''SELECT * 
                     FROM authors
                     LIMIT 5''', con = engine) 

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
pd.io.sql.read_sql('''SELECT * 
                     FROM publishers
                     LIMIT 5''', con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
pd.io.sql.read_sql('''SELECT * 
                     FROM ratings
                     LIMIT 5''', con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [8]:

pd.io.sql.read_sql('''SELECT * 
                     FROM reviews
                     LIMIT 5''', con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


##  Найдем ответы на задания

###  Посчитайте, сколько книг вышло после 1 января 2000 года

In [9]:
pd.io.sql.read_sql('''SELECT COUNT(book_id) 
                     FROM books
                     WHERE EXTRACT(YEAR FROM CAST(publication_date AS date)) > 1999
                     ''', con = engine) 

,count
0,821


В нашем сервисе 821 книга, написанная после 1999 года. А так как всего 1000 книг, то можно сказать, что наш сервис в основном состоит из новых книг

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [10]:
pd.io.sql.read_sql('''SELECT
                      b.book_id
                     ,b.title
                     ,COUNT(DISTINCT re.review_id) as quantity_reviews
                     ,ROUND(AVG(r.rating), 1) as rating
                     FROM books as b
                     LEFT JOIN ratings AS  r ON b.book_id=r.book_id  
                     LEFT JOIN reviews  AS re ON b.book_id = re.book_id
                     GROUP BY b.book_id, b.title
                     LIMIT 5
                     
                     ''', con = engine) 

,book_id,title,quantity_reviews,rating
0,1,'Salem's Lot,2,3.7
1,2,1 000 Places to See Before You Die,1,2.5
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.7
3,4,1491: New Revelations of the Americas Before C...,2,4.5
4,5,1776,4,4.0


В нашей базе пользователи пишут отзывы и ставят оценки книгам. Оценки сильно отличаются. Количество отзывов у первых 10 книг не превышает 4 штук

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
pd.io.sql.read_sql('''SELECT p.publisher as publisher
                     , COUNT( DISTINCT b.book_id)
                     
                     FROM publishers as p
                     
                     INNER JOIN books AS  b ON p.publisher_id=b.publisher_id  
                     WHERE b.num_pages > 50
                     GROUP BY p.publisher_id
                     ORDER BY COUNT( DISTINCT b.book_id) DESC
                     LIMIT 1
                     ''', con = engine) 

,publisher,count
0,Penguin Books,42


Больше всего книг выпустило издательство Penguin Books	- 42 книги


### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;


In [12]:
pd.io.sql.read_sql(''' WITH chosen AS

                       (SELECT 
                           b.book_id as book
                         , b.author_id as author_id
                         , AVG(r.rating) as rating_book
                        FROM books as b
                        LEFT JOIN ratings as r ON b.book_id=r.book_id
                        GROUP BY b.book_id
                        HAVING COUNT(DISTINCT r.rating_id) >= 50)
                     
                       SELECT   a.author
                               , ROUND(AVG(chosen.rating_book),2) as book_rating
                              
                       FROM chosen
                       LEFT JOIN authors as a ON chosen.author_id=a.author_id   
                       GROUP BY a.author
                       ORDER BY book_rating DESC
                       LIMIT 1

                      
                     ''', con = engine) 

,author,book_rating
0,J.K. Rowling/Mary GrandPré,4.28


На первом месте по оценкам авторы "Гарри Поттера" J.K. Rowling/Mary GrandPré, их рейтинг около 4.3

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
pd.io.sql.read_sql(''' WITH 
                        counter AS 
                        (SELECT   
                        COUNT(re.review_id) as count
                        FROM reviews as re
                        WHERE re.username in
                       
                       (SELECT username
                        FROM  ratings 
                        GROUP BY username
                        HAVING COUNT(DISTINCT rating_id) > 50) 
                        
                        GROUP BY re.username)
                        
                        SELECT ROUND(AVG(count), 2)
                        FROM counter
                           
                    
                     ''', con = engine) 

,round
0,24.33


Те, кто ставят много оценок, пишут и много отзывов. Среднее число отзывов от тех, кто поставил больше 50 оценок - более 24

## Выводы
Мы провели первое короткое исследование базы данных о книжном сервисе. Выделили некоторые особенности представленных книг и поведения пользователей.

Но для настоящего исследования не хватает id  пользователей, дат оставления оценок и отзывов, отдельно таблицы о пользователях - когда какую книги купили, когда зарегистрировались.